### **Definir el directorio de trabajo con Google Colab**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.

In [ ]:
import os
from google.colab import drive 
# Enlazar a la carpeta data de la carpeta compartida en Drive
drive.mount('/content/drive')
working_dir = '/content/drive/MyDrive/DS4A-Team12/data'
os.chdir(working_dir)
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWhX740zFiiuoKmLFVo-DzQPBEeNHB1Gq3LivRbbVN2WUMxpxXbwK0U
Mounted at /content/drive
/content/drive/MyDrive/DS4A-Team12/data


In [ ]:
# Librerias relevantes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import joblib
import json

# Opciones de visualización
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
# Abre el diccionario con la estructura de datos definida
tom_datatypes_file = open('datatypes_dictionaries/tomas_datatypes_low_memo.json', 'r')
tom_datatypes = json.load(tom_datatypes_file)
# Abre el dataframe de todas las tomas
tom = pd.read_parquet('preprocessed_data/tomas.parquet')

# Carga el modelo mínimo de predicción a partir de tomas anterior y trasanterior
clf = joblib.load('models/CD_his_RF3.pickle')

In [5]:
import numpy as np
def predict_set(dataset, model):
  df = dataset.sort_values(['IdBeneficiario', 'EdadMeses'])
  cols_modelo = [
                 'EdadMeses',
                 'ZScorePesoTalla',
                 'ZScoreIMC']
  df = df.dropna(subset=cols_modelo).reset_index(drop=True)
  prof = 2 # Profundidad del modelo
  his_list = list()
  for i in range(prof - 1, len(df)):
    id = df.loc[i, 'IdBeneficiario']
    if (df.loc[i - prof + 1, 'IdBeneficiario'] == id):
      fragmento = df.loc[i - prof + 1: i].reset_index(drop=True).\
        reset_index(drop=False)
      melted = fragmento.melt(id_vars='index', value_vars=cols_modelo)
      melted['varname'] = melted.\
        apply(lambda x: x['variable'] + '-' + str(prof - x['index']), axis='columns')
      melted['IdBeneficiario'] = [id] * len(melted)
      pivot = melted.pivot(index='IdBeneficiario', columns='varname', values='value').\
        reset_index(drop=False)
      pivot['EdadMeses-0'] = pivot['EdadMeses-1'] + 3
      his_list.append(pivot)
  his = pd.concat(his_list, axis='index').dropna()
  Ids = his['IdBeneficiario']
  X = his.drop(columns=['IdBeneficiario'])
  y = [q for p, q in model.predict_proba(X)]
  
  prediction = pd.DataFrame({'IdBeneficiario': Ids,
                         'RiesgoDesnutricion3Meses': y})
  prediction = prediction.drop_duplicates(subset=['IdBeneficiario'], keep='last')
  data = df.drop_duplicates(subset=['IdBeneficiario'], keep='last')
  result =  data.merge(prediction, on='IdBeneficiario', how='left')
  return result
 
data = tom[500:600]
predict_set(data, clf)

,IdToma,Registro,Vigencia,Toma,Servicio,FechaValoracionNutricional,EdadMeses,FechaMedicionPerimetroBraquial,MedicionPerimetroBraquial,Peso,Talla,ZScoreTallaEdad,ZScorePesoEdad,ZScorePesoTalla,ZScoreIMC,EstadoTallaEdad,EstadoPesoEdad,EstadoPesoTalla,EstadoIMC,Flag,FechaRegistroSaludNutricion,PresentaCarneVacunacion,ControlesCrecimDesarrollo,AntecedentePremadurez,Direccion,IdBeneficiario,Id,RiesgoDesnutricion3Meses
0,503,1747097,2018,4,HCB TRADICIONAL- COMUNITARIO (T),2018-11-01,48.0,NaT,NaN,15.10,100.000000,-0.69,-0.49,-0.09,-0.11,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,2018-11-20 17:50:55,S,1,NaN,Primera Infancia,720754,2954653,0.340407
1,510,1747098,2018,4,SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - I...,2018-11-06,58.0,2018-02-26,15.9,18.50,108.500000,-0.08,0.17,0.27,0.29,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,2018-11-23 16:43:14,S,2,NaN,Primera Infancia,724314,<NA>,0.258088
2,516,1747099,2018,2,HCB TRADICIONAL- COMUNITARIO (T),2018-04-04,49.0,NaT,NaN,15.50,100.000000,-0.95,-0.52,0.11,0.15,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,2018-04-22 10:37:48,S,1,NaN,Primera Infancia,726327,<NA>,0.327024
3,517,257953,2017,3,SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - ...,2017-08-15,54.0,NaT,NaN,15.15,102.000000,-0.99,-0.93,-0.47,-0.49,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,1,NaN,Primera Infancia,734352,<NA>,NaN
4,520,257955,2017,1,HOGARES INFANTILES - INSTITUCIONAL INTEGRAL,2017-02-28,58.0,NaT,NaN,16.00,105.000000,-0.78,-0.81,-0.52,-0.53,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,2,NaN,Primera Infancia,736046,<NA>,NaN
5,522,257956,2017,2,HCB TRADICIONAL- COMUNITARIO (T),2017-05-05,49.0,NaT,NaN,16.00,90.000000,-3.15,-0.18,2.52,2.59,Retraso en talla,Peso adecuado para la edad,Sobrepeso,Sobrepeso,0,NaT,S,1,NaN,Primera Infancia,736365,<NA>,0.351035
6,525,257957,2017,4,HOGARES INFANTILES - INSTITUCIONAL INTEGRAL,2017-11-20,57.0,NaT,NaN,14.90,98.500000,-2.18,-1.38,-0.04,0.11,Retraso en talla,Riesgo de peso bajo para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,2,NaN,Primera Infancia,736673,<NA>,0.252666
7,537,1747101,2018,4,DESARROLLO INFANTIL EN MEDIO FAMILIAR SIN ARRI...,2018-11-14,54.0,2018-11-14,15.5,17.10,106.000000,-0.21,-0.14,-0.04,-0.02,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,2018-11-19 14:49:42,S,1,NaN,Primera Infancia,736996,5899111,0.317238
8,538,265130,2017,2,HCB FAMI-FAMILIAR TRADICIONAL,2017-04-20,28.0,NaT,NaN,14.00,84.300003,-1.55,0.92,2.39,2.62,Riesgo de baja talla,Peso adecuado para la edad,Sobrepeso,Sobrepeso,0,NaT,S,0,NaN,Primera Infancia,763162,<NA>,NaN
9,542,265132,2017,2,HCB TRADICIONAL- COMUNITARIO (T),2017-05-09,35.0,NaT,NaN,12.90,93.000000,-0.41,-0.48,-0.42,-0.39,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,3,NaN,Primera Infancia,782469,<NA>,0.355565


In [6]:
def predict_IdB(IdBeneficiario, model):
  data = tom[tom['IdBeneficiario'] == IdBeneficiario]
  return predict_set(data, model)['RiesgoDesnutricion3Meses'][0]
 
f"{predict_IdB(843166, clf):.0%}"

'32%'